In [1]:
import joblib
import numpy as np
import pandas as pd

from planet.app import Planet, Geo

In [2]:
model = joblib.load('filename.pkl')

In [3]:
def insert_param(frame: pd.Series):
    coord = Geo(frame['place']).coord
    new_data = pd.Series({        
        'Широта': coord['latitude'],
        'Долгота': coord['longitude'],        
    })

    frame = pd.concat([frame, new_data])    
    parametrs = Planet(coord['longitude'], coord['latitude'], frame['date']).get_parametrs()
    
    return pd.concat([frame, pd.Series(parametrs)])

def make_frame(place: str, date: str, period: int) -> pd.DataFrame:

    frame = pd.DataFrame(
        data=pd.date_range(date, periods=period, freq="D").to_list(),
        columns=['date']
    )
    frame['place'] = place
    
    return frame.apply(insert_param, axis=1)

def predict_magnitude(place: str, date: str, period: int):
    frame = make_frame(place, date, period)
    frame['magnitude'] = model.predict(frame.drop(['date', 'place'], axis=1))
    return frame[['date', 'magnitude']]

In [4]:
place = 'Barnaul, Russia'
date = "2022-01-01"  # начальная дата периода
period = 10  # длительность периода

magnitude_frame = predict_magnitude(place, date, period)

In [5]:
magnitude_frame

,date,magnitude
0,2022-01-01,2.009390
1,2022-01-02,2.014516
2,2022-01-03,2.036781
3,2022-01-04,2.041011
4,2022-01-05,2.044411
5,2022-01-06,2.046760
6,2022-01-07,2.048087
7,2022-01-08,2.048557
8,2022-01-09,2.044103
9,2022-01-10,2.032866


In [6]:
magnitude_frame.describe()

,magnitude
count,10.000000
mean,2.036648
std,0.013972
min,2.009390
25%,2.033844
50%,2.042557
75%,2.046173
max,2.048557
